In [8]:
import os
# input_file = 'input_example.txt'
input_file = 'input_full.txt'

with open(input_file) as file:
    lines = file.readlines()
lines = [line.strip().replace('Valve ', '').replace(' has flow rate=', ';').replace(' tunnels lead to valves ', '').replace(' tunnel leads to valve ', '').replace(' ', '') for line in lines]
lines = [line.split(';') for line in lines]
lines = [[line[0], int(line[1]), line[2].split(',')] for line in lines]
lines
valves, rate, next = {}, {}, {}
nb_openable = 0
for line in lines:
    valves[line[0]] = {'rate': line[1], 'next': line[2]}
    rate[line[0]] = line[1]
    next[line[0]] = line[2]
    if line[1] > 0:
        nb_openable += 1

In [9]:
def calculate_distance(v1, v2):
    paths = [[v1]]
    for dist in range(len(next)+1):
        paths_new = []
        for path in paths:
            valve_curr = path[-1]
            for valve_next in next[valve_curr]:
                if valve_next == v2:
                    return dist+1
                if valve_next not in path:
                    path_new = path.copy() + [valve_next]
                    paths_new += [path_new]
        paths = paths_new.copy()

# VIV = VEry Import Valves :-)
viv = ['AA'] + [valve for valve, rate in rate.items() if rate > 0]
combinations = [(v1, v2) for idx, v1 in enumerate(viv) for v2 in viv[idx + 1:]]

distances = {}
for v1, v2 in combinations:
    distances[(v1,v2)] = calculate_distance(v1, v2)

In [16]:
def get_distance(v1, v2):
    if (v1, v2) in distances.keys():
        return distances[(v1,v2)]
    else:
        return distances[(v2, v1)]

def calculate_released(path, minute, released, releasing, part=1):
    global max_minutes, log
    valve_curr = path[-1]
    if len(path) > 1:
        valve_prev = path[-2]
        distance = get_distance(valve_curr, valve_prev) + 1
        if minute + distance > max_minutes:
            released += releasing * (max_minutes - minute)
            if part == 2:
                log += f'{path};{released}\n'
            return released
        else:
            released += releasing * distance
    else:
        distance = 0
        released = 0
        
    minute += distance
    releasing += rate[valve_curr]
    
    next_possible = [v for v in viv if v not in (path)]
    if len(next_possible) == 0:
        released += releasing * (max_minutes - minute)
        if part == 2:
            log += f'{path};{released}\n'
        return released
    else:
        maximum_released = 0
        for valve_next in next_possible:
            maximum_released = max(maximum_released, calculate_released(path+[valve_next], minute, released, releasing, part))
        return maximum_released

In [13]:
# Part 1
log = ''
max_minutes = 30
print(f"Part 1: {calculate_released(['AA'], 0, 0, 0)}")

Part 1: 1584


In [18]:
# Part 2
log = ''
max_minutes = 26
calculate_released(['AA'], 0, 0, 0, part=2)
if os.path.exists('log.csv'):
  os.remove('log.csv')
log_file = open('log.csv', 'a')
log_file.write(log)
log_file.close()

In [ ]:
# The following is in pySpark

import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType

viv = ['AA', 'EK', 'HT', 'MS', 'VW', 'WI', 'TQ', 'UW', 'FG', 'EG', 'WY', 'AP', 'FX', 'KS', 'TG', 'KR']
binary_to_int = f.udf(lambda x: int(x, 2), IntegerType())

df = (
    spark.read.options(delimiter=';').csv("/FileStore/log.csv")
    .withColumnRenamed('_c0', 'path')
    .withColumnRenamed('_c1', 'released')
    .withColumn('released', f.col('released').cast('int'))
)
for v in viv:
    df = df.withColumn(v, f.col('path').contains(v).cast('int').cast('string'))
df = (
    df
    .withColumn('bin', f.concat(*[v for v in viv if v != 'AA']))
    .withColumn('bin', binary_to_int('bin'))
    .select('bin', 'released')
)
left_df = (
    df
    .withColumnRenamed('bin', 'bin_left')
    .withColumnRenamed('released', 'released_left')
)
right_df = (
    df
    .withColumnRenamed('bin', 'bin_right')
    .withColumnRenamed('released', 'released_right')
)
df_result = (
    left_df.join(right_df, on=[(right_df.bin_right.bitwiseAND(left_df.bin_left) == 0)], how='inner')
    .withColumn('released', f.col('released_left') + f.col('released_right'))
)
df_result.agg({'released': 'max'}).collect()[0]